# PySTAC Introduction

This tutorial includes a basic introduction on reading, writing, and creating STAC objects using Pystac.

It is adapted from the tutorials within the [sat-stac repo](https://github.com/sat-utils/sat-stac/blob/master/tutorial-1.ipynb).

It uses an example stac stored in the `../example-catalog` directory along-side this notebook. The example stac has the following format:

```
../example-catalog
├── catalog.json
└── landsat-8-l1
    ├── 2018-05
    │   └── LC80150322018141LGN00.json
    ├── 2018-06
    │   ├── LC80140332018166LGN00.json
    │   └── LC80300332018166LGN00.json
    ├── 2018-07
    │   └── LC80150332018189LGN00.json
    └── collection.json
```

In [ ]:
import pystac

### Working with existing catalogs

Open a root catalog from it's json file

In [ ]:
cat = pystac.Catalog.from_file("../example-catalog/catalog.json")

We can see all elements of the STAC using the `describe` method

In [ ]:
cat.describe()

Each STAC object has links that you can use to traverse the STAC tree

In [ ]:
cat.links

Pystac has several methods that allow you to access links:

In [ ]:
# Get all child links
cat.get_child_links()

or the children directly:

In [ ]:
list(cat.get_children())

In [ ]:
# or a single child by id
cat.get_child("landsat-8-l1")

In [ ]:
# Get a single link by 'rel'
cat.get_single_link("self")

In [ ]:
# Get item links directly within this catalog (there are none for this catalog)
cat.get_item_links()

or the items directly:

In [ ]:
# get item objects
list(cat.get_items())

In [ ]:
# get all items anywhere below this catalog on the STAC tree
list(cat.get_items(recursive=True))

You can access the stac item from a link using the `target` property

In [ ]:
l = cat.get_single_link("child")
print(l)

In [ ]:
print(l.target)

You can convert any stac item to a python dict using the `to_dict` method.

In [ ]:
cat.to_dict(include_self_link=False)

In [ ]:
# get first (and only in this case) sub-catalog
subcat = next(cat.get_children())

In [ ]:
# print some IDs
print("Root Catalog: ", cat.id)
print("Sub Catalog: ", subcat.id)
print("Sub Catalog parent: ", subcat.get_parent().id)

# iterate through child catalogs of the sub-catalog
print("Sub Catalog children:")
for child in subcat.get_children():
    print("    ", child.id)

In [ ]:
print("\n**Items**")
for i in cat.get_items(recursive=True):
    print(i.id)

### Creating new catalogs

You can initialize a new Catalog with an id and a description. Note that by default it sets a new catalog as root.

In [ ]:
# create a Catalog object with JSON
mycat = pystac.Catalog(id="mycat", description="My shiny new STAC catalog")

In [ ]:
mycat.links

### Adding catalogs to catalogs

In [ ]:
# add a new catalog to a root catalog
kitten = pystac.Catalog(
    id="mykitten", description="A child catalog of my shiny new STAC catalog"
)

When you add a child catalog to a parent catalog, the child catalog assumes the root catalog of it's parent. 'Child' and 'parent' links are also added to the parent and child catalogs, respectively.

In [ ]:
kitten.links

In [ ]:
mycat.add_child(kitten)

In [ ]:
kitten.links

In [ ]:
mycat.links

In [ ]:
mycat.describe()

### Adding collections to catalogs

In the next two steps we will work with Pystac Collections and Items. We will pull them out of our example catalog and add them to the new STAC that we have created.

Collections are Catalogs but also include spatial and temporal extents as well as additional properties. 

In [ ]:
# open the Landsat collection
collection = pystac.Collection.from_file(
    "../example-catalog/landsat-8-l1/collection.json"
)
collection

See the spatial and temporal extent of this collection

In [ ]:
collection.extent.to_dict()

In [ ]:
collection.links

In [ ]:
# add it to the child catalog created above
kitten.add_child(collection)

In [ ]:
collection.links

### Adding items to collection

Items are stac objects whose parents can be either Catalogs or Collections. They also have spatio-temporal information and assets. Assets point directly to the data included in the STAC.

In [ ]:
# open a Landsat item
item = pystac.read_file(
    "../example-catalog/landsat-8-l1/2018-05/LC80150322018141LGN00.json"
)
item

In [ ]:
item.links

In [ ]:
item.assets

In [ ]:
# add it to the collection created above
collection.add_item(item)

In [ ]:
# now look at the catalog we've created
mycat.describe()

Currently, this STAC only exists in memory. We can use `normalize_and_save` to save off the STAC with the canonical "absolute published" form:

In [ ]:
mycat.normalize_and_save(
    "pystac-example-absolute", catalog_type=pystac.CatalogType.ABSOLUTE_PUBLISHED
)

Notice now that the 'parent' link of an item is a absolute HREF:

In [ ]:
item = next(mycat.get_items(recursive=True))
item.get_single_link("parent").get_href()

We can also normalize and save the catalog to the other types described in the best practices documentation: "relative published" and "self contained". A self contained catalog contains all relative links, and no self links. Notice how saving a self contained catalog will produce relative links:

In [ ]:
mycat.normalize_and_save(
    "pystac-example-relative", catalog_type=pystac.CatalogType.SELF_CONTAINED
)

In [ ]:
item = next(mycat.get_items(recursive=True))
item.get_single_link("parent").get_href()